In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

In [1]:
import requests
from langchain_core.tools import tool
# 지역명 location 으로 위도 경도 알아내는 api
# location='서울시 구로구'
@tool    # 함수 테스트하고 싶으면 tool 데코레이터 주석처리 하기
def  get_coordinates(location):
  """
    주어진 장소 이름을 기반으로 위도와 경도를 조회합니다.
    OpenStreetMap Nominatim API를 사용합니다.
  """
  url = f"https://nominatim.openstreetmap.org/search?q={location}&format=json"
  headers = {
    "User-Agent": "LangChainApp/1.0 (your@email.com)"   # ⬅ http 요청 헤더:서버에게 요청을 보낸 클라이언트의 정보
  }
  res = requests.get(url, headers=headers).json()  # res 는 요청에 대한 응답
  if res:
    # print(res)  # 응답 출력
    lat = res[0]['lat']
    lon = res[0]['lon']
    print(f"{location}의 위도는 {lat}, 경도는 {lon}입니다.")
    return f"{lat},{lon}"
  else:
    # print("좌표를 찾을 수 없습니다.")
    return "좌표를 찾을 수 없습니다."



In [3]:
get_coordinates('서울시 구로구')    # @tool 은 직접 호출할 수 없음.(TypeError: 'StructuredTool' object is not callable)

서울시 구로구의 위도는 37.4951999, 경도는 126.8877000입니다.


'37.4951999,126.8877000'

In [3]:
import os
import json

# @tool  # description 속성 또는 아래와 같은 docstring 작성해야 함.
def get_weather_info(lat_lon):
  """
    get_coordinates 함수에서 리턴한 위도와 경도가 있는 문자열을 전달 받아서
    api.openweathermap.org API 를 사용하여 날씨를 구합니다.
  """
  # lat_lon = '37.5666791,126.9782914'
  lat, lon = lat_lon.split(',')
  print(lat,lon)
  api_key = os.getenv('OPENWEATHER_API_KEY')
  url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=kr"
  res = requests.get(url).json()
  if 'weather' in res:
      weather_desc = res['weather'][0]['description']
      temp = res['main']['temp']
      feels_like = res['main']['feels_like']
      humidity = res['main']['humidity']
      
      result = (
        f"날씨: {weather_desc}\n" 
        f"기온(°C): {temp}°C\n"
        f"체감온도(°C): {feels_like}°C\n"
        f"습도(%): {humidity}%\n"
        f"지역: {res.get('name', 'Unknown')}"
        )
      
      # 결과는 문자열로 리턴
      return result
  else:
      # print("날씨 정보를 불러올 수 없습니다.")
      return "날씨 정보를 불러올 수 없습니다."

get_weather_info('37.4951999,126.8877000')

37.4951999 126.8877000


'날씨: 맑음\n기온(°C): 9.98°C\n체감온도(°C): 7.33°C\n습도(%): 29%\n지역: Kwangmyŏng'

In [ ]:
# store 없이 단일 기록만 유지
history = InMemoryChatMessageHistory()

def get_session_history(_: str):  # 세션 ID 무시
    return history

llm_message_history = RunnableWithMessageHistory(llm, get_session_history)
